# Find N-termini that are (de)stabilized by UBR1 KO

jyelland

## Procedure

1. Clean data from [Timms et al](https://pubmed.ncbi.nlm.nih.gov/31273098/).

## Load libraries

In [ ]:
library(tidyverse)
library(readxl)
library(rasilabRtemplates)

## Load data

In [ ]:
psi_values <- read_excel("../../data/2024/aaw4912_timms_sm_data-file-s3.xlsx", sheet = 1, skip = 1) %>%
# select columns of interest
    select(1,2,10,18,26,34) %>%
# clean names
    janitor::clean_names() %>%
    print()

## Calculate ∆PSI values

In [ ]:
delta_psi <- psi_values %>%
# make each KO a unique observation
    pivot_longer(cols = 3:6) %>%
# assign control condition ko = 0
    mutate(ko = if_else(!str_detect(name,"aavs"), parse_number(name), 0)) %>%
    select(-name) %>%
# group by transcript ID because for some reason, there are duplicate gene names
    group_by(ensembl_transcript_id) %>%
# subtract control psi from KO psis to calculate ∆PSI
    mutate(d_value = value - value[ko == 0]) %>%
    select(-value) %>%
# organize by transcript and gene for nice printing 
    pivot_wider(id_cols = c(ensembl_transcript_id, gene_symbol), names_from = ko, names_prefix = "ko_", values_from = d_value,
        names_expand = TRUE) %>%
    print()

## Plot data

In progress...

In [ ]:
plot_data <- delta_psi %>%
    mutate(mean_effect = mean(c(ko_1,ko_2)))

plot_data %>% ggplot(aes(x = mean_effect, y = -log10(ko_3))) +
    geom_point()

## Print table of N-termini _destabilized_ by UBR1
These are presumed to be UBR1 substrates with ∆PSI above 0.8

In [ ]:
delta_psi %>%
# filter to ∆PSI values > 0.8, per Timms et al.
    filter(ko_1 > 0.8) %>%
    filter(ko_2 > 0.8) %>%
# determine UBR1-specific effect by filtering absolute magnitude of KO 3
# (not sure how this was done in Timms et al.)
    filter(abs(ko_3) < 0.8) %>%
# calculate mean stabilizing effect of clone 1 and clone 2 KO
    mutate(mean_effect = mean(c(ko_1,ko_2))) %>%
    arrange(desc(mean_effect)) %>%
# write table
    write_tsv("../../data/2024/ubr1_destabilized.csv") %>%
    print()


## Print table of N-termini _stabilized_ by UBR1
These N-termini may be more stable in the presence of UBR1, for whatever reason

In [ ]:
delta_psi %>%
# filter to ∆PSI values > 0.8, per Timms et al.
    filter(ko_1 < -0.8) %>%
    filter(ko_2 < -0.8) %>%
# determine UBR1-specific effect by filtering absolute magnitude of KO 3
# (not sure how this was done in Timms et al.)
    filter(abs(ko_3) < 0.8) %>%
# calculate mean stabilizing effect of clone 1 and clone 2 KO
    mutate(mean_effect = mean(c(ko_1,ko_2))) %>%
    arrange(mean_effect) %>%
# write table
    write_tsv("../../data/2024/ubr1_stabilized.csv") %>%
    print()
